## Model Training


In [78]:
# Importing Libraries
import numpy as np
import pandas as pd


In [79]:
df = pd.read_csv('data/adult.csv')
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [80]:
#space remover
def space_remover(dataframe):
   for i in dataframe.columns:
      if dataframe[i].dtype == 'O':
         dataframe[i] = dataframe[i].map(str.strip)
      else:
         pass
space_remover(df)

In [81]:
df.drop_duplicates(keep='first',inplace=True)

In [82]:
salary_map = {'<=50K' :0 ,'>50K':1}

df['salary'] = df['salary'].map(salary_map)

In [83]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [84]:
df.replace('?',np.nan,inplace=True)

In [85]:
df.drop(labels=['fnlwgt','capital_loss'],axis=1,inplace=True)

In [86]:
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,country,salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,40,Cuba,0


In [87]:
X = df.iloc[:,:-1]
y = df['salary']

In [88]:
X.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,country
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,40,United-States
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,13,United-States
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,40,United-States
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,40,United-States
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,40,Cuba


In [89]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: salary, dtype: int64

In [90]:
# segregate Num and Cat features

categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

print(f'Categorical columns:{categorical_cols}')
print(f'Numerical columns:{numerical_cols}')

Categorical columns:Index(['workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'country'],
      dtype='object')
Numerical columns:Index(['age', 'education_num', 'capital_gain', 'hours_per_week'], dtype='object')


In [91]:
#model training
from sklearn.impute import SimpleImputer #Handling missing values
from sklearn.preprocessing import StandardScaler #handling Feature Scaling
from sklearn.preprocessing import OneHotEncoder # Encoding
##pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [92]:
##Numerical pipeline
num_pipeline = Pipeline(
   steps=[
      ('imputer',SimpleImputer(strategy='median')),
      ('scaler',StandardScaler())
   ]
)
##Categorical pipeline
cat_pipeline = Pipeline(
   steps=[
      ('imputer',SimpleImputer(strategy='most_frequent')),
      ('one-hot-encoder',OneHotEncoder(sparse_output=False)),
      ('scaler',StandardScaler())
   ]
)

preprocessor = ColumnTransformer([
   ('num_pipeline',num_pipeline,numerical_cols),
   ('cat_pipeline',cat_pipeline,categorical_cols)
]
)

In [93]:
#train tset split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [94]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())

X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [95]:
X_train.head()

,num_pipeline__age,num_pipeline__education_num,num_pipeline__capital_gain,num_pipeline__hours_per_week,cat_pipeline__workclass_Federal-gov,cat_pipeline__workclass_Local-gov,cat_pipeline__workclass_Never-worked,cat_pipeline__workclass_Private,cat_pipeline__workclass_Self-emp-inc,cat_pipeline__workclass_Self-emp-not-inc,...,cat_pipeline__country_Portugal,cat_pipeline__country_Puerto-Rico,cat_pipeline__country_Scotland,cat_pipeline__country_South,cat_pipeline__country_Taiwan,cat_pipeline__country_Thailand,cat_pipeline__country_Trinadad&Tobago,cat_pipeline__country_United-States,cat_pipeline__country_Vietnam,cat_pipeline__country_Yugoslavia
0,1.498616,1.132773,-0.142996,1.588788,-0.174114,-0.259899,-0.012804,0.570946,-0.191245,-0.289168,...,-0.032659,-0.061181,-0.018109,-0.049231,-0.042015,-0.023959,-0.023087,0.310204,-0.045764,-0.021236
1,0.323774,-0.033504,-0.142996,-0.031707,-0.174114,-0.259899,-0.012804,0.570946,-0.191245,-0.289168,...,-0.032659,-0.061181,-0.018109,-0.049231,-0.042015,-0.023959,-0.023087,0.310204,-0.045764,-0.021236
2,-1.438489,-0.033504,-0.142996,-1.652201,-0.174114,-0.259899,-0.012804,0.570946,-0.191245,-0.289168,...,-0.032659,-0.061181,-0.018109,-0.049231,-0.042015,-0.023959,-0.023087,0.310204,-0.045764,-0.021236
3,0.103491,-0.422263,-0.142996,-0.031707,-0.174114,-0.259899,-0.012804,0.570946,-0.191245,-0.289168,...,-0.032659,-0.061181,-0.018109,-0.049231,-0.042015,-0.023959,-0.023087,0.310204,-0.045764,-0.021236
4,-0.704213,-1.588540,-0.142996,-0.679904,-0.174114,-0.259899,-0.012804,0.570946,-0.191245,-0.289168,...,-0.032659,-0.061181,-0.018109,-0.049231,-0.042015,-0.023959,-0.023087,0.310204,-0.045764,-0.021236


In [96]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [97]:
#Evaluate Model

def evaluate_model(true,pred):
   acc = accuracy_score(true,pred)
   return acc

In [98]:
models = {
   'LogisticRegression':LogisticRegression(),
   'Decision Tree':DecisionTreeClassifier(),
   'Random Forest':RandomForestClassifier(),
   'Support vector Machine':SVC(),
   'KNN':KNeighborsClassifier()
}
model_list = []
acc_list=[]

for i in range(len(list(models))):
   model = list(models.values())[i]
   model.fit(X_train,y_train)

   #make prediction
   y_pred = model.predict(X_test)

   acc = evaluate_model(y_test,y_pred)

   print(list(models.keys())[i])
   model_list.append(list(models.keys())[i])

   print('Model Traning Performance')
   print('Accuracy_score',acc)

   acc_list.append(acc)

   print('='*25)
   print('\n')




LogisticRegression
Model Traning Performance
Accuracy_score 0.8518746158574063


Decision Tree
Model Traning Performance
Accuracy_score 0.8087277197295636


Random Forest
Model Traning Performance
Accuracy_score 0.8392132759680393


Support vector Machine
Model Traning Performance
Accuracy_score 0.8458512599877074


KNN
Model Traning Performance
Accuracy_score 0.8221266133988937




In [99]:
print('classification_report: ','\n',classification_report(y_test,y_pred))
print('\n')
print('confusion_matrix: ','\n',confusion_matrix(y_test,y_pred))

classification_report:  
               precision    recall  f1-score   support

           0       0.87      0.90      0.88      6159
           1       0.65      0.58      0.61      1976

    accuracy                           0.82      8135
   macro avg       0.76      0.74      0.75      8135
weighted avg       0.82      0.82      0.82      8135



confusion_matrix:  
 [[5546  613]
 [ 834 1142]]
